#  Implementação de um Agente em Flappy Bird

### Imports

In [1]:
import gym
from gym import spaces
import gym_ple
import numpy as np

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


## Flappy Bird Sem Visão Computacional

### Ambiente

Modelagem do ambiente do Flappy Bird usando o GameState como observação.

#### Observação:
 * Posição Y do pássaro.
 * Velocidade Y do pássaro.
 * Distância do pássaro até o próximo cano.
 * Posição Y da parte de cima do próximo cano.
 * Posição Y da parte de baixo do próximo cano.
 * Distância do pássaro até o cano depois do próximo cano.
 * Posição Y da parte de cima do cano depois do próximo cano.
 * Posição Y da parte de baixo do cano depois do próximo cano.

In [2]:
class FlappyBirdEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array']}

  def __init__(self):
    self.n_pipes = 0
    self.env = gym.make("FlappyBird-v0")
    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions:
    self.action_space = self.env.action_space
    # Example for using image as input:
    self.observation_space = spaces.Box(low=np.array([0, -10.0, 0, 0, 0, 0, 0, 0]),
                                        high=np.array([512, 10.0, 588.0, 512, 512, 588.0, 512, 512]),
                                        dtype=np.float32)

  def step(self, action):
    observation, reward, done, info = self.env.step(action)
    observation = np.array(list(self.env.game_state.getGameState().values()))
    if reward > 0:
        self.n_pipes += 1
    if done:
        reward = -1
    reward += 0.1
    reward += (75 - abs(observation[0] - (observation[3] + observation[4])/2))*max((300 - observation[2])/300, 0)/750
    
    if self.n_pipes >= 500:
        done = True
    
    return observation, reward, done, info

  def reset(self):
    self.env.reset()
    self.n_pipes = 0
    observation = np.array(list(self.env.game_state.getGameState().values()))
    return observation  # reward, done, info can't be included

  def render(self, mode='human'):
    self.env.render(mode='human')
    
  def close (self):
    self.env.close()

Checando se o nosso ambiente satisfaz as propriedades do Gym.

In [3]:
from stable_baselines.common.env_checker import check_env

env = FlappyBirdEnv()
check_env(env)

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Criando um vetor com 16 ambientes para multiprocessamento.

In [4]:
import gym

from stable_baselines.common import set_global_seeds, make_vec_env
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize

env = make_vec_env(FlappyBirdEnv, n_envs = 8)
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

### Agente

Criando o modelo de PPO2 com a biblioteca Stable Baselines.

In [5]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines import PPO2

model = PPO2(MlpPolicy, env, n_steps = 1024, learning_rate=0.0001, nminibatches = 64, lam = 0.95, gamma = 0.99, noptepochs= 15, ent_coef= 0.0, cliprange= 0.2, verbose=1)





Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Você tem a opção de usar um modelo pré-treinado:

In [12]:
from stable_baselines import PPO2

model = PPO2.load("trained_models/PPO2_1280")
model.set_env(env)

Loading a model without an environment, this model cannot be trained until it has a valid environment.


Criando um Callback

In [6]:
from stable_baselines.common.callbacks import EvalCallback

EvalEnv = make_vec_env(FlappyBirdEnv, n_envs = 1)
EvalEnv = VecNormalize(EvalEnv, norm_obs=True, norm_reward=False, clip_obs=10.)

eval_callback = EvalCallback(EvalEnv, best_model_save_path='./eval_models/',
                             log_path='./logs/', n_eval_episodes = 10,
                             eval_freq=1024,deterministic=True, render=False)

D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


#### Treinamento do Modelo:

In [8]:
# model.learn(total_timesteps=65536, callback=eval_callback)
model.learn(total_timesteps=131072, callback=eval_callback)

Eval num_timesteps=8192, episode_reward=128.81 +/- 75.41
Episode length: 723.10 +/- 405.35
New best mean reward!
--------------------------------------
| approxkl           | 0.0028525698  |
| clipfrac           | 0.022379557   |
| ep_len_mean        | 156           |
| ep_reward_mean     | 23.1          |
| explained_variance | 0.212         |
| fps                | 110           |
| n_updates          | 1             |
| policy_entropy     | 0.4604574     |
| policy_loss        | -0.0061988872 |
| serial_timesteps   | 1024          |
| time_elapsed       | 0             |
| total_timesteps    | 8192          |
| value_loss         | 1.9070802     |
--------------------------------------
Eval num_timesteps=16384, episode_reward=88.62 +/- 65.62
Episode length: 504.30 +/- 347.81
-------------------------------------
| approxkl           | 0.0026084033 |
| clipfrac           | 0.016267903  |
| ep_len_mean        | 180          |
| ep_reward_mean     | 27.2         |
| explained_variance 

Eval num_timesteps=106496, episode_reward=1698.82 +/- 318.99
Episode length: 9157.40 +/- 1717.57
New best mean reward!
-------------------------------------
| approxkl           | 0.0019077639 |
| clipfrac           | 0.014168294  |
| ep_len_mean        | 639          |
| ep_reward_mean     | 112          |
| explained_variance | -0.418       |
| fps                | 11           |
| n_updates          | 13           |
| policy_entropy     | 0.3629764    |
| policy_loss        | -0.002405172 |
| serial_timesteps   | 13312        |
| time_elapsed       | 1.95e+03     |
| total_timesteps    | 106496       |
| value_loss         | 2.2577927    |
-------------------------------------
Eval num_timesteps=114688, episode_reward=1487.93 +/- 668.05
Episode length: 8020.20 +/- 3587.53
--------------------------------------
| approxkl           | 0.0016605406  |
| clipfrac           | 0.011173503   |
| ep_len_mean        | 693           |
| ep_reward_mean     | 122           |
| explained_varianc

#### Testando o Modelo: 

In [52]:
import itertools

obs = EvalEnv.reset()
dones = 0
for t in itertools.count():
    EvalEnv.render()
    action, _states = model.predict(obs)
    obs, rewards, dones, info = EvalEnv.step(action)
    if dones:
        break
    
print(t)
# print(EvalEnv.n_pipes)
EvalEnv.close()

1602


#### Salvando o Modelo:

In [49]:
model.save("trained_models/NormPPO2")

#### Gravando um Episódio: (Not Working)

Criando o ambiente gravado.

In [11]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

video_folder = 'videos/'
video_length = 100

VideoEnv = DummyVecEnv([lambda: FlappyBirdEnv()])

obs = VideoEnv.reset()

VideoEnv = VecVideoRecorder(VideoEnv, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix="FlappyBird")

Rodando o episódio.

In [13]:
obs = VideoEnv.reset()

for _ in range(video_length + 1):
    VideoEnv.render()
    action, _ = model.predict(obs)
    obs, _, _, _ = VideoEnv.step(action)

VideoEnv.close()


D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Env returned None on render(). Disabling further rendering for video recorder by marking as disabled: path=D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.mp4 metadata_path=D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.meta.json
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Saving video to  D:\Codigos\RL\FlappyBirdPPO2\videos\FlappyBird-step-101-to-step-201.mp4


## Flappy Bird com Visão Computacional

A implementar

In [11]:
ENV_NAME = "FlappyBird-v0"

import gym

from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

env_vision = gym.make(ENV_NAME)
# Optional: PPO2 requires a vectorized environment to run
# the env is now wrapped automatically when passing it to the constructor
env_vision = DummyVecEnv([lambda: env_vision])

In [ ]:
modelVision = PPO2(CnnLstmPolicy, env_vision, n_steps = 512, nminibatches = 1, lam = 0.98, gamma = 0.999, noptepochs= 15, ent_coef= 0.01, verbose=1)

In [ ]:
modelVision.learn(total_timesteps=10000)

In [ ]:
obs = env_vision.reset()
dones = 0
while not dones:
    action, _states = modelVision.predict(obs)
    obs, rewards, dones, info = env_vision.step(action)
    env.render()

env_vision.close()